# Convolutional Neural Networks with Keras (2)

Instead of starting from scratch, we can make use of the many available pre-trained networks.

In general, alternatives when using a pre-trained network are

- Use the complete network - that is, use the final predictions - for the new data. This only makes sense when the new data have classes included in the original dataset!
- Use the bottleneck features (activations from last MaxPooling layer before fully connected layers) and train a fully-connected network on top.


We have follow path 2, because we're introducing our own classification task.

Steps:

1) instantiate convolutional part of the model (everything up to the fully-connected layers) and run this model on the training and test data once, recording the output (the "bottleneck features", i.e. the last activation maps before the fully-connected layers) in two numpy arrays

2) train a small fully-connected model on top of the stored features

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import numpy as np
import os

Using TensorFlow backend.


In [2]:
img_width, img_height = 375,500

train_data_dir = 'data/train'
test_data_dir = 'data/test'

n_train_samples = 235
n_train_ants = 114
n_train_bees = 121


n_test_samples = 148
n_test_ants = 66
n_test_bees = 82

num_epochs = 50
batch_size = 16

top_model_weights_path = 'bottleneck_fc_model.h5'
bottleneck_features_train_path = "bottleneck_features_train.npy"
bottleneck_features_test_path = "bottleneck_features_test.npy"

# even the single pass through VGG16 takes a long time, so we don't do this "live"
bottleneck_features_exist = True

# save time here, too
top_model_exists = True

### Step 1: save bottleneck features

We store the features offline for performance reasons - VGG16 is a resource-intensive model!

In [3]:
# load the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [5]:
# no data augmentation this time
datagen = ImageDataGenerator(rescale=1. / 255)

# training data
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, # no labels
        shuffle=False)

Found 235 images belonging to 2 classes.


In [6]:
if not bottleneck_features_exist:
    bottleneck_features_train = model.predict_generator(train_generator, n_train_samples//batch_size+1, verbose=1)
    np.save(bottleneck_features_train_path, bottleneck_features_train)

In [7]:
# test data
test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)


Found 148 images belonging to 2 classes.


In [8]:
if not bottleneck_features_exist:
    bottleneck_features_test = model.predict_generator(test_generator, n_test_samples//batch_size+1, verbose=1)
    np.save(bottleneck_features_test_path, bottleneck_features_test)

### Step 2: load saved data and train a small fully-connected model on top

In [9]:
train_data = np.load(bottleneck_features_train_path)
train_labels = np.array([0] * n_train_ants + [1] * n_train_bees)
train_labels.shape, train_data.shape

((235,), (235, 11, 15, 512))

In [10]:
test_data = np.load(bottleneck_features_test_path)
test_labels = np.array([0] * n_test_ants + [1] * n_test_bees)
test_labels.shape, test_data.shape

((148,), (148, 11, 15, 512))

In [11]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 84480)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               21627136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 21,627,393
Trainable params: 21,627,393
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
if not top_model_exists:
    model.fit(train_data, train_labels,
              epochs=num_epochs,
              batch_size=batch_size,
              validation_data=(test_data, test_labels))
    model.save_weights(top_model_weights_path)

### Test the model?

At this time, there is no overall model that could be used for new predictions.

This will be addressed in the next notebook.